## Estructura de la Aplicación:

- **Verificación del backup:**
    - Detectar la presencia del archivo DBSIF.FBK.
    - Verificar la fecha y hora de creación/modificación.
    - Obtener el tamaño del archivo.

- **Notificación por Email y/o WhatsApp:**
    - Si el backup es exitoso (el archivo cumple con los criterios), se envía una notificación.
    - En caso de fallo (si no se genera el archivo o tiene un tamaño inesperado), enviar una alerta.

**Requisitos:**

- Bibliotecas necesarias:
    - `os` para verificar el archivo.
    - `smtplib` para enviar emails.
    - API de WhatsApp (podemos usar Twilio o alguna otra plataforma que permita la integración).
    - `schedule` o utilizar una tarea programada en Windows para ejecutar la verificación.

## Verificación del archivo DBSIF.FBK:


In [3]:
import os
import time

def verificar_backup(ruta_backup):
    # Verificar si el archivo existe
    if os.path.exists(ruta_backup):
        # Obtener el tamaño en GB
        tamano = os.path.getsize(ruta_backup) / (1024 * 1024 * 1024)
        # Obtener fecha de creación/modificación
        timestamp = os.path.getmtime(ruta_backup)
        fecha_creacion = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(timestamp))

        return {"existe": True, "tamano_gb": tamano, "fecha_creacion": fecha_creacion}
    else:
        return {"existe": False}



## Envío de notificaciones por correo electrónico:

In [4]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

def enviar_email(asunto, cuerpo, destinatario):
    remitente = "tu_email@gmail.com"
    contraseña = "tu_contraseña"
    
    # Configuración del mensaje
    mensaje = MIMEMultipart()
    mensaje['From'] = remitente
    mensaje['To'] = destinatario
    mensaje['Subject'] = asunto
    mensaje.attach(MIMEText(cuerpo, 'plain'))

    # Conectar y enviar
    servidor = smtplib.SMTP('smtp.gmail.com', 587)
    servidor.starttls()
    servidor.login(remitente, contraseña)
    texto = mensaje.as_string()
    servidor.sendmail(remitente, destinatario, texto)
    servidor.quit()

    print("Correo enviado a", destinatario)


## Envío de notificaciones por WhatsApp: Usando la API de Twilio:

In [ ]:
# pip install twilio

from twilio.rest import Client

def enviar_whatsapp(mensaje):
    account_sid = 'tu_account_sid'
    auth_token = 'tu_auth_token'
    client = Client(account_sid, auth_token)

    message = client.messages.create(
        body=mensaje,
        from_='whatsapp:+14155238886',  # Número de Twilio
        to='whatsapp:+tu_numero'
    )
    print("WhatsApp enviado:", message.sid)

## Integración: Unir la verificación del backup con el envío de notificaciones:

In [ ]:
def verificar_y_notificar():
    resultado = verificar_backup('ruta/del/backup/DBSIF.FBK')

    if resultado['existe']:
        mensaje = f"Backup exitoso.\nTamaño: {resultado['tamano_gb']:.2f} GB\nFecha de creación: {resultado['fecha_creacion']}"
        enviar_email("Backup Exitoso", mensaje, "destinatario@ejemplo.com")
        enviar_whatsapp(mensaje)
    else:
        mensaje = "ERROR: No se ha generado el archivo de backup."
        enviar_email("Backup Fallido", mensaje, "destinatario@ejemplo.com")
        enviar_whatsapp(mensaje)

verificar_y_notificar()
